In [ ]:
import numpy as np
import pandas as pd
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Creating Dictionary for guidelines

In [ ]:
recycling_instruction_dataset = [
    {'location_id':1,
    'location':"Cook County, IL",
     'instructions': [
         {
         "instruc_id":0,
         "items": "metal",
         "instructions": "Metal objects that are able to be recycled includes steel and aluminum containers and foil."
        },
         {"instruc_id":1,
         "items": "paper",
         "instructions": "paper that can be recycled includes cardboard (flattened), office paper, newspaper, and magazines"
          },
         {
          "instruc_id":2,
         "items": "glass",
         "instructions": "glass items that can be recycled includes glass bottles and jars only"
         },
         {
          "instruc_id":3,
         "items": "plastic",
         "instructions": """Plastic items that can be recycled includes plastic bottles, tubs, jugs, and jars only.
          Always replace your cap onto the empty and clean plastic bottle it came from.
          Then, place into your recycling bin to ensure both plastic components are successfully processed at your local recycling facility."""
         },
         {
          "instruc_id":4,
         "items": "coffee cups",
         "instructions": """Your cup of joe is a recycling no. Lids, straws, and hot beverage cups should be kept out of your curbside recycling bin.
          Despite the paper composition of hot beverage cups, they contain a lining that degrades the overall quality of the recycling batch and should not be recycled in your bin.
          For iced coffee, some uncertainty remains about whether the plastic cup is recyclable. Please check with your local authority to determine whether plastic coffee cups can go in your bin.
          To reduce waste and save resources, use a reusable coffee cup and straw! Some coffee establishments even offer discounts when you bring your own cup/mug."""
         },
         {
          "instruc_id":5,
         "items": "other",
         "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
         }
         ]
     },
    {
      'location_id':2,
      'location':"Palm Desert, CA",
      'instructions': [
         {
         "instruc_id":0,
         "items": "metal and glass",
         "instructions": """The following metal and glass items can be recycled:
         -Aluminum, steel, and tin cans,
         -Clean aluminum foil,
        - Glass bottles and jars (remove lids)"""
        },
         {"instruc_id":1,
         "items": "paper",
         "instructions": """
         The following paper items can be recycled:
        - White and colored paper,
        - Phone books,
        - Magazines,
        - Junk Mail,
        - Envelopes,
        - File folders,
        - Cardboard (flattened),
        - Cartons (milk, juice, & broth),
        - Books (remove hard covers or binding),
        - Cardboard boxes, e.g. cereal, etc.
         """
          },
         {
          "instruc_id":2,
         "items": "plastic",
         "instructions": """Plastic items that can be recycled includes:
          - Styrofoam packing blocks
          - Plastic medicine bottles (empty)
          - Plastic bottles numbered 1-7 (remove plastic film or wrap)"""
         },
         {
          "instruc_id":3,
         "items": "other naps",
         "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
         }
         ]
    }
]

In [ ]:
results_df = pd.read_csv('/content/drive/My Drive/capstone_annie/llava_knn_k1_results_10_27.csv')

In [ ]:
classes_col = np.repeat(np.arange(1, 8), 2)
loc_name = np.tile(['Cook County, IL', 'Palm Desert, CA'], 7)
loc_idx = np.tile([1, 2], 7)
mapping_to_instruc = [0, 0, 0, 0, 4, 3, 1, 1, 2, 0, 3, 2, 5, 3]

instruc_mapping_df = pd.DataFrame({'pred_class': classes_col, 'location': loc_name, 'location_id': loc_idx, 'instruction_id': mapping_to_instruc})

In [ ]:
[x['location_id'] for x in recycling_instruction_dataset]

[1, 2]

In [ ]:
results_df['new_target_wording'] = np.where(results_df['target'] == 'aluminum_can', "tin can", "other")
results_df.loc[results_df['target'] == 'paper_cup', 'new_target_wording'] = "paper cup"
results_df.loc[results_df['target'] == 'boxes', 'new_target_wording'] = "cardboard boxes"
results_df.loc[results_df['target'] == 'plastic_bottle', 'new_target_wording'] = "plastic bottle"
results_df.loc[results_df['target'] == 'glass_bottle', 'new_target_wording'] = "glass bottle"

In [ ]:
pred_class = 2
loc_choice = 1

instruc_mapping_df[(instruc_mapping_df['pred_class']==pred_class) & (instruc_mapping_df['location_id']==loc_choice)]['instruction_id']

,instruction_id
2,0


In [ ]:
results_df.loc[10]['output'][2:-2]

'Bushman waste canister'

In [ ]:
raw_outputs = []
class_names_list = []
location_idxes = []
retr_instrucs = []
pred_class_list = []

locations_list = [x['location_id'] for x in recycling_instruction_dataset]

for i in range(len(results_df)):

  loc_choice = random.randint(1, 2)
  location_idxes.append(loc_choice)

  raw_llava_output = results_df.loc[i]['output'][2:-2]
  raw_outputs.append(raw_llava_output)

  fixed_wording = results_df.loc[i]['new_target_wording']
  class_names_list.append(fixed_wording)

  loc_idx = locations_list.index(loc_choice)

  loc_dict = recycling_instruction_dataset[loc_idx]

  instructions_dict_list = loc_dict['instructions']

  instruc_id_list = [x['instruc_id'] for x in instructions_dict_list]

  pred_class = results_df.loc[i]['final_selected_class']
  pred_class_list.append(pred_class)
  # print(instruc_mapping_df[(instruc_mapping_df['pred_class']==pred_class) & (instruc_mapping_df['location_id']==loc_choice)]['instruction_id'])
  relevant_sec_id = instruc_mapping_df[(instruc_mapping_df['pred_class']==pred_class) & (instruc_mapping_df['location_id']==loc_choice)]['instruction_id'].values[0]

  instruc_idx = instruc_id_list.index(relevant_sec_id)

  retrieve_instruc = instructions_dict_list[instruc_idx]['instructions']
  retr_instrucs.append(retrieve_instruc)

stage_2_inputs = pd.DataFrame({'llava_outputs': raw_outputs, 'class_names': class_names_list, 'loc_idx': location_idxes, 'retr_instrucs': retr_instrucs, 'pred_class':pred_class_list})

In [ ]:
stage_2_inputs.to_csv('/content/drive/My Drive/capstone_annie/stage_2_inputs_10_27.csv')

In [ ]:
stage_2_inputs

,llava_outputs,class_names,loc_idx,retr_instrucs,pred_class
0,Aluminum can,tin can,2,The following metal and glass items can be rec...,1
1,Aluminum can,tin can,1,Metal objects that are able to be recycled inc...,1
2,Aluminum can,tin can,2,The following metal and glass items can be rec...,1
3,Aluminum can,tin can,1,Metal objects that are able to be recycled inc...,1
4,Aluminum can,tin can,2,The following metal and glass items can be rec...,1
...,...,...,...,...,...
524,Plastic bottle,glass bottle,2,Plastic items that can be recycled includes:\n...,6
525,Plastic bottle,glass bottle,1,Plastic items that can be recycled includes pl...,6
526,Plastic bottle,glass bottle,1,Plastic items that can be recycled includes pl...,6
527,Alcohol bottle,glass bottle,2,The following metal and glass items can be rec...,5


In [ ]:
results_df.loc[76]

,76
Unnamed: 0.1,380
Unnamed: 0,380
output,['Plastic waste item made from plastic.']
neighbor,plastic bottle
distance,1.105135
final_selected_class,7
id,76
target,paper_cup
true_class,3
new_target_wording,paper cup


In [ ]:
stage_2_inputs.loc[80]

,80
llava_outputs,Plastic cup
class_names,paper cup
loc_idx,1
retr_instrucs,Plastic items that can be recycled includes pl...
pred_class,6


# Bin Classification Results

In [ ]:
results_df

,Unnamed: 0.1,Unnamed: 0,output,neighbor,distance,final_selected_class,id,target,true_class,new_target_wording
0,0,0,['Aluminum can'],aluminum food can,0.680598,1,0,aluminum_can,1,tin can
1,5,5,['Aluminum can'],aluminum food can,0.680598,1,1,aluminum_can,1,tin can
2,10,10,['Aluminum can'],aluminum food can,0.680598,1,2,aluminum_can,1,tin can
3,15,15,['Aluminum can'],aluminum food can,0.680598,1,3,aluminum_can,1,tin can
4,20,20,['Aluminum can'],aluminum food can,0.680598,1,4,aluminum_can,1,tin can
...,...,...,...,...,...,...,...,...,...,...
524,2620,2620,['Plastic bottle'],plastic bottle,0.325713,6,524,glass_bottle,5,glass bottle
525,2625,2625,['Plastic bottle'],plastic bottle,0.325713,6,525,glass_bottle,5,glass bottle
526,2630,2630,['Plastic bottle'],plastic bottle,0.325713,6,526,glass_bottle,5,glass bottle
527,2635,2635,['Alcohol bottle'],alcohol bottle,0.350173,5,527,glass_bottle,5,glass bottle
